<a href="https://colab.research.google.com/github/NavyaS26/Internship/blob/main/Fashion_mnist_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Add, Activation, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


import seaborn as sns

In [16]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train_flat = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test_flat = x_test.reshape(-1, 784).astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [17]:
def create_resnet_dnn():
    inputs = layers.Input(shape=(784,))

    x = layers.BatchNormalization()(inputs)
    # Initial dense layer
    x = layers.Dense(784, activation='relu')(inputs)

    # Residual block 1
    x1 = layers.Dense(784, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(x)
    x1 = layers.Dropout(0.6)(x1)
    x2 = layers.Dense(784, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(x1)
    res1 = layers.Add()([x, x2])  # Skip connection
    res1 = layers.Activation('relu')(res1)
    res1 = layers.Dropout(0.5)(res1)

    # Residual block 2
    x3 =layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(res1)
    x3 = layers.Dropout(0.4)(x3)
    x4 = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(x3)
    # Need to match dimensions for the skip connection
    res1_down = layers.Dense(1024, activation='relu')(res1)  # Projection to match dimensions
    res2 = layers.Add()([res1_down, x4])
    res2 = layers.Activation('relu')(res2)
    res2 = layers.Dropout(0.4)(res2)


    # Residual block 3
    x5 = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(res2)
    x5 = layers.Dropout(0.3)(x5)
    x6 = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-5))(x5)
    res2_down = layers.Dense(512, activation='relu')(res2)
    res3 = layers.Add()([res2_down, x6])
    res3 = layers.Activation('relu')(res3)
    res3 = layers.Dropout(0.3)(res3)


    # Final layers
    x_final = layers.Dense(256, activation='relu')(res3)
    x_final = layers.Dense(128, activation='relu')(x_final)
    x_final = layers.Dense(64, activation='relu')(x_final)
    outputs = layers.Dense(10, activation='softmax')(x_final)

    return Model(inputs, outputs)

In [18]:
resnet = create_resnet_dnn()

resnet.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
lr_schedule = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True)


callbacks=[early_stop, lr_schedule]
history_resnet = resnet.fit(x_train_flat,
                    y_train,
                    validation_split=0.3,
                    epochs=20,
                    batch_size=128,
                    callbacks=callbacks,
                    verbose=1
                    )

Epoch 1/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 55s 167ms/step - accuracy: 0.8884 - loss: 0.3400 - val_accuracy: 0.8793 - val_loss: 0.3870 - learning_rate: 0.0010
Epoch 2/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 53s 161ms/step - accuracy: 0.8915 - loss: 0.3302 - val_accuracy: 0.8803 - val_loss: 0.3756 - learning_rate: 0.0010
Epoch 3/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 83s 163ms/step - accuracy: 0.8934 - loss: 0.3164 - val_accuracy: 0.8784 - val_loss: 0.3860 - learning_rate: 0.0010
Epoch 4/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 86s 175ms/step - accuracy: 0.8995 - loss: 0.3072 - val_accuracy: 0.8772 - val_loss: 0.3699 - learning_rate: 0.0010
Epoch 5/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 78s 162ms/step - accuracy: 0.9002 - loss: 0.2998 - val_accuracy: 0.8852 - val_loss: 0.3709 - learning_rate: 0.0010
Epoch 6/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 81s 158ms/step - accuracy: 0.9028 - loss: 0.3008 - val_accuracy: 0.8749 - val_loss: 0.3839 - learning_rate: 0.0010
Epoch 7/20
329/329 ━━━━━━━━━━━━━━━━━━━━ 87s 173ms/step - accuracy: 0.8

In [22]:
resnet_test_loss, resnet_test_acc = resnet.evaluate(x_test_flat, y_test, verbose=0)
print(f"ResNet-style DNN Test Accuracy: {resnet_test_acc:.4f}")

ResNet-style DNN Test Accuracy: 0.8828
